O multiplicador de investimentos é uma medida que indica quanto o produto interno bruto (PIB) de um país aumenta em resposta a um aumento nos investimentos. Ele é calculado com base na Margem de Propensão a Investir (MPI) e na Margem de Propensão a Importar (MPI).

A fórmula geral para calcular o multiplicador de investimentos é:

Multiplicador de Investimentos
​

$$ \text{Multiplicador de Investimentos} = \frac{1}{1 - (MPI \times MPM)} $$


Onde:

* MPI é a Margem de Propensão a Investir, que representa a proporção do aumento nos investimentos que é gasto em bens e serviços domésticos.
* MPM é a Margem de Propensão a Importar, que representa a proporção do aumento na renda nacional que é gasta em importações.
Para calcular o multiplicador de investimentos, você precisaria conhecer os valores específicos da MPI e do MPM. Esses valores podem ser obtidos a partir de dados econômicos, como a taxa de investimento e a taxa de importação.

A MPI geralmente é calculada como a relação entre o aumento nos investimentos e o aumento no PIB, enquanto o MPM é a relação entre as importações e o PIB.

Depois de calcular esses valores, você pode inseri-los na fórmula acima para encontrar o multiplicador de investimentos específico para a economia em questão.

A MAI fornece informações sobre como os investimentos afetam a produção e o emprego em diferentes setores da economia. Por outro lado, a MIP mostra as relações entre os diferentes setores da economia em termos de produção, consumo e insumos.

Para estimar uma proxy para a MPI a partir da MAI, você pode considerar a parcela dos investimentos totais que são absorvidos pela produção doméstica. Isso pode ser calculado como a razão entre os investimentos absorvidos internamente e o total de investimentos. Isso lhe dará uma ideia de quanto dos investimentos são gastos na economia doméstica, em vez de serem gastos em importações.

Da mesma forma, para estimar uma proxy para a MPM a partir da MIP, você pode considerar a parcela dos gastos em importações em relação à produção total de cada setor. Isso pode ser calculado como a razão entre os gastos em importações e o total da produção de cada setor. Isso lhe dará uma ideia de quanto da produção de cada setor é gasta em importações.

Essas estimativas podem não ser perfeitamente precisas, mas podem fornecer uma aproximação útil para a MPI e a MPM, que você pode usar para calcular um proxy para o multiplicador de investimentos.

# Cálculo do proxy MPI

In [181]:
import pandas as pd
import numpy as np
#periodos 
#lista_periodo = [2013,2014,2015,2016,2017,2018,2019]
lista_periodo = [2019]
#Calcular a demanda total (imp + nacional) e a demanda total nacional
df_producao = pd.DataFrame()
for periodo in lista_periodo:
    V = pd.read_excel('./dados/MIP_'+str(periodo)+'_67_PCE.xlsx',sheet_name='Recursos',skiprows=5,skipfooter=4,usecols=[*range(11,78)],header=None)
    V = np.transpose(V)
    g = np.matmul(V,np.ones(126)) ## Produção total por atividade No caso da MIP estimada, o transporte aquaviário foi fundido com o terrestre de cargas
    q = np.transpose(np.matmul(np.transpose(np.ones(67)),V)) ## Produção total por produto
    D = np.matmul(V,np.linalg.inv(np.diag(q))) # market share

    
    #Consumo intermediário produção importados
    Um = pd.read_excel('./dados/MIP_2019_67_PCE.xlsx',sheet_name='Usos Importado',skiprows=5,skipfooter=4,usecols=[*range(2,69)],header=None)
    qm = np.transpose(np.matmul(np.transpose(np.ones(67)),Um.transpose())) ## Produção total por produto do consumo intermediário importado
    qm = pd.DataFrame(qm)
    Dm = np.nan_to_num(Um.to_numpy()/qm.to_numpy()).transpose()
    

    #Demanda Final importados
    Fm = pd.read_excel('./dados/MIP_2019_67_PCE.xlsx',sheet_name='Usos Importado',skiprows=5,skipfooter=4,usecols=[*range(70,76)],header=None)
    fm = np.matmul(Fm,np.transpose(np.ones(6))) # soma da demanda final
    ef = np.matmul(Dm,np.diag(fm)) #distribuição da demanda final importada na mesma proporção do market share dos produtos importados




/tmp/ipykernel_449/107957505.py:20: RuntimeWarning: invalid value encountered in divide
  Dm = np.nan_to_num(Um.to_numpy()/qm.to_numpy()).transpose()


In [195]:
#Demanda total importada por setor e produto
Dem_total = (Um + ef.transpose()).transpose()

In [219]:
V.shape

(67, 126)

In [230]:
#Demanda total ( consumo intermediário + demanda final ) nacional + importada por produto
np.matmul(np.ones(67),V).to_numpy() + np.matmul(np.ones(67),Dem_total).to_numpy()

array([  19989.,   44525.,   18394.,   57068.,  153601.,   69633.,
         10031.,   19525.,   29994.,   74643.,   31966.,   10717.,
         34607.,   26424.,   17783.,   13385.,   21434.,  256572.,
        107234.,   23444.,  150742.,   18631.,   68786.,    8174.,
         23779.,   64980.,   33244.,   33325.,   81882.,   13368.,
         21318.,   45242.,   40232.,  137038.,  103287.,   25913.,
         14145.,   24270.,   33442.,   81193.,   45865.,   33930.,
         35925.,   78042.,   19481.,   20222.,  103274.,   22185.,
         22895.,  147578.,  224267.,   87288.,   82906.,   62140.,
         79230.,   71017.,   71318.,   40191.,   20479.,   64149.,
        118781.,   40939.,  101222.,   12942.,   22406.,   59474.,
         22655.,  144317.,   83211.,    6221.,  130724.,   36075.,
         42180.,   88965.,   28291.,   96630.,   27026.,   36536.,
         31099.,  150190.,  160739.,   67554.,  142599.,   84171.,
         44899.,   57707.,  104474.,  346873.,   95105.,  3301

In [231]:
#Demanda total ( consumo intermediário + demanda final ) nacional + importada por setor 

Dem_total_nac_imp = np.matmul(V,np.ones(126)).to_numpy() + np.matmul(Dem_total,np.ones(126)).to_numpy()

In [234]:
#Participacao da importacao na produção total por setor
participacao_importacao_por_setor = np.matmul(Dem_total,np.ones(126)).to_numpy()/Dem_total_nac_imp

In [ ]:
def retorna_indice_setor(setor,multiplicador):
    lista_setores = ["Agricultura, inclusive o apoio à agricultura e a pós-colheita","Pecuária, inclusive o apoio à pecuária","Produção florestal; pesca e aquicultura","Extração de carvão mineral e de minerais não metálicos","Extração de petróleo e gás, inclusive as atividades de apoio","Extração de minério de ferro, inclusive beneficiamentos e a aglomeração","Extração de minerais metálicos não ferrosos, inclusive beneficiamentos","Abate e produtos de carne, inclusive os produtos do laticínio e da pesca","Fabricação e refino de açúcar","Outros produtos alimentares","Fabricação de bebidas","Fabricação de produtos do fumo","Fabricação de produtos têxteis","Confecção de artefatos do vestuário e acessórios","Fabricação de calçados e de artefatos de couro","Fabricação de produtos da madeira","Fabricação de celulose, papel e produtos de papel","Impressão e reprodução de gravações","Refino de petróleo e coquerias","Fabricação de biocombustíveis","Fabricação de químicos orgânicos e inorgânicos, resinas e elastômeros","Fabricação de defensivos, desinfestantes, tintas e químicos diversos","Fabricação de produtos de limpeza, cosméticos/perfumaria e higiene pessoal","Fabricação de produtos farmoquímicos e farmacêuticos","Fabricação de produtos de borracha e de material plástico","Fabricação de produtos de minerais não-metálicos","Produção de ferro-gusa/ferroligas, siderurgia e tubos de aço sem costura","Metalurgia de metais não ferosos e a fundição de metais","Fabricação de produtos de metal, exceto máquinas e equipamentos","Fabricação de equipamentos de informática, produtos eletrônicos e ópticos","Fabricação de máquinas e equipamentos elétricos","Fabricação de máquinas e equipamentos mecânicos","Fabricação de automóveis, caminhões e ônibus, exceto peças","Fabricação de peças e acessórios para veículos automotores","Fabricação de outros equipamentos de transporte, exceto veículos automotores","Fabricação de móveis e de produtos de indústrias diversas","Manutenção, reparação e instalação de máquinas e equipamentos","Energia elétrica, gás natural e outras utilidades","Água, esgoto e gestão de resíduos","Construção","Comércio por atacado e varejo","Transporte terrestre","Transporte aquaviário","Transporte aéreo","Armazenamento, atividades auxiliares dos transportes e correio","Alojamento","Alimentação","Edição e edição integrada à impressão","Atividades de televisão, rádio, cinema e  gravação/edição de som e imagem","Telecomunicações","Desenvolvimento de sistemas e outros serviços de informação","Intermediação financeira, seguros e previdência complementar","Atividades imobiliárias","Atividades jurídicas, contábeis, consultoria e sedes de empresas ","Serviços de arquitetura, engenharia, testes/análises técnicas e P & D","Outras atividades profissionais, científicas e técnicas","Aluguéis não imobiliários e gestão de ativos de propriedade intelectual","Outras atividades administrativas e serviços complementares","Atividades de vigilância, segurança e investigação","Administração pública, defesa e seguridade social","Educação pública","Educação privada","Saúde pública","Saúde privada","Atividades artísticas, criativas e de espetáculos","Organizações associativas e outros serviços pessoais","Serviços domésticos"]
    indice = lista_setores.index(setor)
    if multiplicador == "producao":
        return mul_prod[indice]
    elif multiplicador == "salario":
        return mul_sal[indice]
    elif multiplicador == "salario ajustado":
        return mul_sal_ajus[indice]
    elif multiplicador == "impostos":
        return mul_imp[indice]
    elif multiplicador == "ocupações":
        return mul_ocup[indice]
